# BetterTransformer
- 추론 속도를 빠르게하는 방법
- Encoder, EncoderLayer, MultiHeadAttention에 Kernel Fusion과 Nested tensor을 활용한 Transformer구조
    - Kernel Fusion : 두개 이상의 kerenl들을 합치는 것. 독립적인 계산들을 하나로 통합하여 메모리 이동 최소화
    - Nested tensor : 차원마다 다른 크기를 가진 tensor. 입력의 희소성을 활용하여 패딩 토큰에 불필요한 작업을 하지 않도록 함

In [ ]:
!pip install accelerate
!pip install optimm
!pip install transformers==4.39.3

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch
from transformers import (
    PretTrainedTokenizerFast,
    BartForConditionalGeneration
)

model_id = "Sehong/kobart-QuestionGeneration"

tokenizer = PretTrainedTokenizerFast.from_pretrianed(model_id)

model = BartForConditionalGeneration.from_pretrianed(
    model_id,
    torch_dtype="auto",
    device_map="auto"
)

model.to_bettertransformer()

In [ ]:
messages = """<s>
안녕하세요.
임직원의 폭풍성장을 돕는 패스트캠퍼스 기업교육팀입니다.
기술 발전과 다양서으이 증대로 인해 리더십의 개념이 크게 변화하며,
과거 위계질서 중심의 리더십에서 벗어난 협업과소통 중심의 리더십이 새롭게 각광받고 있죠.
이러한 변화의 흐름 속에서 리더들이 팀원들을 효과적으로 읶르고,
성공적인 비지ㅣ스를 이루기 우해서는 알맞은 리더십 교육의 제공이 매우 중요합니다.
다시 말해, HRD의 역할이 그 어느 때보다 중용하다는 의미죠.
리더십 교육에 대해 고민하는 HRD 여러분을 위해 패스트캠퍼스 기업교육팀이 준비했습니다.
하단의 아티클을 통해 리더로서 성공하기 위해 갖춰야할 핵심 리더십 역량이 무엇인지 확인해보세요.
도한, 직무 맞춤형 리더십 교육의 설계부터 평가까지, 한 번에 해결할 수 있는 무료 템플릿도 받아보세요.
이번 레터를 통해 담당잔님들의 리더십 교육에 대한 고민 해결에 도움이 되길 바랍니다.
감사합니다. </s>"""

In [ ]:
input_ids = tokenizer.encode(
    messages,
    return_tensors="pt"
).repeat(50,1).to(model.device)

In [ ]:
outputs = model.generate(
    input_ids,
    max_length=100,
    do_sample=True
)

In [ ]:
for i in range(10):
    print(tokenizer.decode(outputs[i], skip_special_tokens=True))